<a href="https://colab.research.google.com/github/tsakailab/prml/blob/master/ipynb/ex_MNIST_LogisticRegression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MNIST画像のロジスティック回帰（logistic regression on MNIST datasets）

MNIST画像の識別問題において，ロジスティック回帰で学習した単純パーセプトロンの重みを可視化します．

----

氏名：

学生番号：

----
## 基本課題（必須）

    1. 「★ロジスティック回帰を実行します．」で，損失の係数`C`を小さい値に設定し，L2正則化の効果を強くすると，
       訓練データ（training data）と検証データ（validation data）の正答率はどう変わりますか．
       また，パーセプトロンの重みはどうなりますか．

（ここに回答を書いてください）



    2. 「★学習した重みを可視化し，識別関数を計算します．」を実行すると，
       (a)識別に役立つ特徴が学習によって見つけられていること，(b)識別的特徴に基づき識別関数が計算されていること  
       を観察できます．表示される図や数値をどのように見ると(a)と(b)を確認できますか．

（ここに回答を書いてください）



    3. 手書き数字画像以外のデータ集合（例えばFashion-MNIST）で2つのクラスを選択し，2クラス識別器を作成してください．
       パーセプトロンの重みから，その2つのクラスにはどのような特徴の違いが見つかりましたか．
       また，「★ロジスティック回帰を実行します．」において，どのような設定でLogisticRegression()を使用すると，
       選択した2クラスの識別的特徴が考察しやすいですか．

（ここに回答を書いてください）



    4.その他，気づいたこと，調べたことがあれば書いてください．

（ここに回答を書いてください）



----
発展課題（任意）がこのノートブックの後半にあります．

### データ集合を取得します．

In [ ]:
from torchvision import datasets

# [MNIST](http://yann.lecun.com/exdb/mnist/)
tvds = datasets.MNIST('./data', download=True, train=True)

# [Fashion-MNIST](https://github.com/zalandoresearch/fashion-mnist)
#tvds = datasets.FashionMNIST('./data', download=True, train=True)

# [Kuzushiji-MNIST](https://github.com/rois-codh/kmnist)
#tvds = datasets.KMNIST('./data', download=True, train=True)

Ximages_all, y_all = tvds.data.numpy(), tvds.targets.numpy()

# [EMNIST](https://pytorch.org/vision/stable/generated/torchvision.datasets.EMNIST.html)
#tvds = datasets.EMNIST('./data', download=True, train=True, split='letters')
#Ximages_all, y_all = tvds.data.transpose_(-1,-2).numpy(), tvds.targets.numpy() - 1

In [ ]:
#@title ☆画像数とサイズを確認します（画像を加工したい場合はこのセルを編集して利用してください）．

# simulate noisy images
#Ximages_all = Ximages_all + np.random.rand(*Ximages_all.shape) * 200
#Ximages_all[Ximages_all > 255] = 255

import numpy as np
from skimage.filters import gaussian
from skimage.exposure import equalize_hist as eh
from skimage.transform import resize

'''
* blurring (https://scikit-image.org/docs/stable/api/skimage.filters.html#skimage.filters.gaussian)
* histogram equalization (https://scikit-image.org/docs/stable/auto_examples/color_exposure/plot_equalize.html)
* resize images (https://scikit-image.org/docs/stable/auto_examples/transform/plot_rescale.html)
'''
#Ximages_all = gaussian(np.float32(Ximages_all.transpose((1,2,0))), sigma=1.0, multichannel=True).transpose(2,0,1)
#Ximages_all = eh(Ximages_all.transpose((1,2,0))).transpose(2,0,1) * 255
#height, width = 8, 8; Ximages_all = resize(Ximages_all.transpose((1,2,0)), (height, width)).transpose(2,0,1)

Ximages_all = np.uint8(Ximages_all)
print("(#images, height, width)", Ximages_all.shape)
print(Ximages_all.dtype, ", max. pixel value = ", Ximages_all.max())

MNIST class labels

| [MNIST](http://yann.lecun.com/exdb/mnist/) | 0 | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 |
| - | :-: | :-: | :-: | :-: | :-: | :-: | :-: | :-: | :-: | :-: |
| [Fashion-MNIST](https://github.com/zalandoresearch/fashion-mnist) | T-shirt/top | Trouser | Pullover | Dress | Coat | Sandal |  Shirt | Sneaker | Bag | Ankle boot |
| [Kuzushiji-MNIST](https://github.com/rois-codh/kmnist) | お | き | す | つ | な | は |  ま | や | れ | を |

In [ ]:
#@title 画像を例示します．
#@title Show images of digits
import numpy as np
from matplotlib import pyplot as plt
print("%d images in total" % len(y_all))

# show the digits
def plotMNIST(imgs, gts, preds=None, num=8, mag=0.6):
    classes = np.unique(gts)
    nc = len(classes)
    idx = np.arange(len(gts))
    fig = plt.figure(figsize=(num*mag, nc*mag))  # figure size in inches
    fig.subplots_adjust(left=0, right=1, bottom=0, top=1, hspace=0.05, wspace=0.05)
    for i, c in enumerate(classes):
        imgsc = imgs[gts==c]
        idc = idx[gts==c]
        for j in range(min(num,len(imgsc))):
            ax = fig.add_subplot(nc, num, i*num+j+1, xticks=[], yticks=[])
            ax.imshow(imgsc[j], cmap=plt.cm.gray, interpolation='nearest')

            # label the image with the target value
            ax.text(0, imgs.shape[1]*0.2, str(c), color='white')
            if preds is not None:
                if preds[idc[j]] == c:
                    ax.text(imgs.shape[2]*0.8, imgs.shape[1]*0.2, str(preds[idc[j]]), color='#a0ffa0')
                else:
                    ax.text(imgs.shape[2]*0.8, imgs.shape[1]*0.2, str(preds[idc[j]]), color='red')

p = np.random.permutation(len(y_all))
plotMNIST(Ximages_all[p], y_all[p], num=16)

### 2クラスの識別問題にしたい場合（さもなくば実行しなくてよいです）
- このセルの `pos` と `neg` でクラスを指定して実行してください．

In [ ]:
c = 2
pos = 1 # choose from 0 to 9
neg = 0 # choose from 0 to 9

Ximages = Ximages_all[np.logical_or(y_all == pos, y_all == neg)]
y = y_all[np.logical_or(y_all == pos, y_all == neg)]
yp, yn = y == pos, y== neg
y[yp] = 1
y[yn] = 0
lbl = [neg, pos]

p = np.random.permutation(len(y))
plotMNIST(Ximages[p], y[p], num=20)

### すべてのクラスを使いたい場合

In [ ]:
c = len(np.unique(y_all)) # len(np.unique(y_all))+1 # 10
lbl = range(c)
Ximages = Ximages_all
y = y_all

### 訓練データと検証データに分けます．
- `n_train` と `n_val` で訓練データと検証データの数をそれぞれ指定できます．

In [ ]:
n_train = 3000
n_val = 1000

from sklearn.model_selection import train_test_split
# https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

# split the data into training and validation sets
Ximages_train, Ximages_val, y_train, y_val = train_test_split(Ximages, y, 
                                                              train_size=n_train, test_size=n_val)

# reshape to 28*28=784-dimensional feature vectors
X_train = np.reshape(Ximages_train, (Ximages_train.shape[0], -1)) / 255   # (n_train, 784)
X_val = np.reshape(Ximages_val, (Ximages_val.shape[0], -1)) / 255         # (n_val, 784)
print("X_train.shape = ", X_train.shape)
print("X_val.shape = ", X_val.shape)

### ★ロジスティック回帰を実行します．
- [sklearn.linear_model.LogisticRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html)を使用します．引数の仕様とデフォルトの値を確認してください．
- 実装されているロジスティック損失は訓練データ数で除された平均ではありません．[損失の係数`C`](https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression)で設定できます．

In [ ]:
from sklearn.linear_model import LogisticRegression

# 反復数以外はデフォルトの値で実行します．
model =  LogisticRegression(max_iter=1000)

# 損失の係数`C`を小さい値に設定し，L2正則化の効果を強くします．
#model =  LogisticRegression(C=100/n_train, max_iter=1000, penalty='l2')

# [Elastic net regularization](https://en.wikipedia.org/wiki/Elastic_net_regularization) i.e., L1+L2 regularization
#model =  LogisticRegression(C=100/n_train, penalty='elasticnet', l1_ratio=0.5, solver='saga')

# sparse regularization
#model =  LogisticRegression(penalty='l1', solver='saga')

model.fit(X_train, y_train)

In [ ]:
#@title 識別の結果を例示します（上：訓練データ，下：検証データ）．
%matplotlib inline
from sklearn.metrics import accuracy_score

y_pred = model.predict(X_train)
print("Accuracy on training data: ", accuracy_score(y_train, y_pred))
p = np.random.permutation(n_train)
plotMNIST(Ximages_train[p], y_train[p], y_pred[p], num=16)
plt.show()

y_pred = model.predict(X_val)
print("\nAccuracy on validation data: ", accuracy_score(y_val, y_pred))
p = np.random.permutation(n_val)
plotMNIST(Ximages_val[p], y_val[p], y_pred[p], num=16)

MNIST class labels

| [MNIST](http://yann.lecun.com/exdb/mnist/) | 0 | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 |
| - | :-: | :-: | :-: | :-: | :-: | :-: | :-: | :-: | :-: | :-: |
| [Fashion-MNIST](https://github.com/zalandoresearch/fashion-mnist) | T-shirt/top | Trouser | Pullover | Dress | Coat | Sandal |  Shirt | Sneaker | Bag | Ankle boot |
| [Kuzushiji-MNIST](https://github.com/rois-codh/kmnist) | お | き | す | つ | な | は |  ま | や | れ | を |

In [ ]:
#@title ★学習した重みを可視化し，識別関数を計算します．

w = np.c_[model.intercept_, model.coef_]
#print(w.shape)

# Plot the weights
fig = plt.figure(figsize=(12, 3))  # figure size in inches
fig.subplots_adjust(left=0, right=1, bottom=0, top=1, hspace=0.05, wspace=0.05)
cmap = plt.cm.seismic

from matplotlib.colors import TwoSlopeNorm as tsn
#import warnings
#warnings.filterwarnings("ignore")#, category=np.VisibleDeprecationWarning)

jval = np.random.randint(n_val,size=1) # draw one test sample
vmin = w[:,1:].min()
vmax = -vmin
xmax=max(X_val[jval,:].ravel())

if c == 2:
    c = 1
for k in range(c):
  
    ax = fig.add_subplot(3, c, k + 1, xticks=[], yticks=[])
    norm = tsn(vmin=np.minimum(w[k,1:].min(),-1e-6), vcenter=0, vmax=np.maximum(w[k,1:].max(),1e-6))
    ax.imshow(w[k,1:].reshape(Ximages.shape[1], Ximages.shape[2]), vmin=vmin,vmax=vmax, cmap=cmap, norm=norm)
    if c == 1:
        ax.text(0, 4, str(lbl[c]) + ' ?', color='k')
    else:
        ax.text(0, 4, str(lbl[k]) + ' ?', color='k')

    ax = fig.add_subplot(3, c, k + c + 1, xticks=[], yticks=[])
    #ax.imshow(X_val[jval,:].reshape(Ximages.shape[1], Ximages.shape[2]), cmap=plt.cm.gray)
    ax.imshow(Ximages_val[jval].squeeze(), cmap=plt.cm.gray)
    ax.text(0, 7, str(lbl[y_val[jval][0]]), color='white')

    wXk = X_val[jval,:].ravel()*w[k,1:]
    ax = fig.add_subplot(3, c, k + 2*c + 1, xticks=[], yticks=[])
    norm = tsn(vmin=np.minimum(wXk.min(),-1e-6), vcenter=0, vmax=np.maximum(wXk.max(),1e-6))
    ax.imshow(wXk.reshape(Ximages.shape[1], Ximages.shape[2]), vmin=vmin*xmax, vmax=vmax*xmax, cmap=cmap, norm=norm)
    g = wXk.sum()+w[k,0]
    if g >= 0:
        color='r'
    else:
        color='b'
    ax.text(0, 4, str(lbl[model.predict(X_val)[jval][0]]), color=color)
    ax.text(0, 33, '$g=$'+'{:.1f}'.format(g), fontsize=max(10+g,10), color=color)

可視化の説明：
- <b>上段</b>：パーセプトロンの入力の画素値に対応する重みを図示しています．
    - 赤が正，青は負 の重みを表します．
    - 2クラスの場合は，正と負がそれぞれ`pos`と`neg`に寄与する重みです．
- <b>中段</b>：入力画像です．画素値は非負で，黒はゼロです．
- <b>下段</b>：パーセプトロンの重み（上段）と入力画像（中段）の画素値の積を図示しています．
- 各クラスのパーセプトロンが計算した識別関数 <i>g</i> を各列の下に表示しています．

In [ ]:
#@title 混同行列（行：正解，列：予測）
from sklearn import metrics
# https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html

cm = metrics.confusion_matrix(y_val, y_pred)
print(cm)

print(cm.sum(axis=0))

In [ ]:
#@title おまけ
import seaborn as sns

# Make predictions on test data
cm = metrics.confusion_matrix(y_val, y_pred)
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

plt.figure(figsize=(9,9))
sns.heatmap(cm_normalized, annot=True, fmt=".3f", linewidths=.5, square = True, cmap = 'Blues_r');
plt.ylabel('Actual label');
plt.xlabel('Predicted label');

--------
## 発展課題（任意）

[MedMNIST](https://github.com/MedMNIST/MedMNIST)を使ってましょう．
手書き数字画像のように手軽に使える実験用の医用画像です．  
下の2つのセルを実行後は，このJupyter Notebook前半の  
「☆画像数とサイズを確認します（画像を加工したい場合はこのセルを編集して利用してください）．」  
以降を実行できます．

    1. PneumoniaMNISTは胸部X線CT画像です．'0': 正常, '1': 異常（肺炎）の2クラス識別問題です．
       学習したパーセプトロンは，主に画像のどこの画素値をどのように重みづけしますか．

（ここに回答を書いてください）



    2. MedMNISTにはどのような医用画像の識別問題がありますか．

（ここに回答を書いてください）[参考文献](https://arxiv.org/abs/2110.14795)





In [ ]:
#@title [MedMNIST](https://github.com/MedMNIST/MedMNIST)を取得します．
!pip install -q medmnist
import medmnist
print(f"MedMNIST v{medmnist.__version__} @ {medmnist.HOMEPAGE}")

In [ ]:
# 'pneumoniamnist', 'chestmnist', 'octmnist', 'breastmnist', 'tissuemnist', 'organamnist', 'organcmnist', 'organsmnist' 
data_flag = 'pneumoniamnist' 
DataClass = getattr(medmnist, medmnist.INFO[data_flag]['python_class'])

tvds = DataClass(split='train', download=True)
#tvds = DataClass(split='test', download=True)

print(tvds)
Ximages_all, y_all = tvds.imgs, tvds.labels.ravel()

お疲れさまでした．